In [1]:
import pandas as pd
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/drive.metadata.readonly"]


def createClient():
  """Shows basic usage of the Drive v3 API.
  Prints the names and ids of the first 10 files in the specified folder.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("drive", "v3", credentials=creds)
    return service
    
  except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  drive = createClient()

In [3]:
# Call the Drive v3 API
try: 
    results = (
        drive.files()
        .list(pageSize=50, q="'1FPWkaHGWfEuJJBSFrd2p2DPnfg_Jujtf' in parents", fields="files(id, name)")
        .execute()
    )
    items = results.get("files", [])

    if not items:
      print("No files found.")
except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f"An error occurred: {error}")

In [4]:
items

[{'id': '1McUiBy7lFXZVfnLrQXKWsRW7V0MivDWZ',
  'name': "NYCMelee's Stock Exchange #29"},
 {'id': '1Lu-0f5BWJF7FV1cp2SKqeOfCj0gdoc8u',
  'name': "NYCMelee's Stock Exchange #1"},
 {'id': '1LH87AeM90E-1mYCKWcelooxBOm9iIiqQ',
  'name': "NYCMelee's Stock Exchange #2"},
 {'id': '1YgWw8ifnWnovTq7LO8YNqom4LaknsU7D',
  'name': "NYCMelee's Stock Exchange #3"},
 {'id': '1f2vD30-mUs54qPnnwDbbpXwZOw8NRLOw',
  'name': "NYCMelee's Stock Exchange #4"},
 {'id': '1Ugt4nlhEWAnxnKXSC41L4CVUcYYqvbGq',
  'name': "NYCMelee's Stock Exchange #5"},
 {'id': '1T2ZYs-y1e8xsFrd4HsFA1gk9ld0rpJ-4',
  'name': "NYCMelee's Stock Exchange #8"},
 {'id': '1IdtpAVo8cmugQa9GzPrtfdy6_nOI6V2h',
  'name': "NYCMelee's Stock Exchange #9"},
 {'id': '1hqI0jJvXEuD-eLr_-RhDd9eGltZBfT5M',
  'name': "NYCMelee's Stock Exchange #18"},
 {'id': '1SzO1_tieM-XrUfiPsQxwmwOBj4czjYYM',
  'name': "NYCMelee's Stock Exchange #17"},
 {'id': '13tBxgxW8_D7vXKpyaFj9CIm41HhXdvsk',
  'name': "NYCMelee's Stock Exchange #19"},
 {'id': '1HME93NbPcUeHJLNQg2

In [5]:
file_holder = []

In [6]:
for item in items:
    try: 
        results = (
            drive.files()
            .list(pageSize=50, q=f"'{item['id']}' in parents", fields="files(name, webContentLink)")
            .execute()
        )
        replays = results.get("files", [])

        if not replays:
            print("No files found.")
            print("Files:")
        for replay in replays:
            if 'webContentLink' in replay:
                replay['tournament'] = item['name']
                file_holder.append(replay)
    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f"An error occurred: {error}")

In [7]:
replays_df = pd.DataFrame(file_holder)
replays_df.head()

,name,webContentLink,tournament
0,Redemption Bracket LSF - J Solo (Marth) vs Bre...,https://drive.google.com/uc?id=1Ta64MD-DQuoEPL...,NYCMelee's Stock Exchange #29
1,Redemption Bracket LQF - Combusting (Falco) vs...,https://drive.google.com/uc?id=1jKE0tqoIEBM2uc...,NYCMelee's Stock Exchange #29
2,Redemption Bracket LR5 - Mister (Falco) vs Emp...,https://drive.google.com/uc?id=1M825xiFvbXH7Oc...,NYCMelee's Stock Exchange #29
3,Redemption Bracket LR5 - Franco (DK) vs Breakf...,https://drive.google.com/uc?id=17z9ioG4eevR6zO...,NYCMelee's Stock Exchange #29
4,Redemption Bracket LR4 - Franco (DK) vs JKille...,https://drive.google.com/uc?id=1HKlXQAjBeVtd-m...,NYCMelee's Stock Exchange #29


In [8]:
replays_df.to_json("./replays.json", orient="records")